In [ ]:
import tensorflow as tf
from tensorflow.python.training.moving_averages import assign_moving_average
import numpy as np
import os, pdb
import cv2
import numpy as np
import random as rn
import tensorflow as tf
import threading
import time

global n_classes,layer_count
n_classes = 50
layer_count = 0

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
gpu_options = tf.GPUOptions(allow_growth = True)

In [ ]:
#==========================================================================
#=============Reading data in multithreading manner========================
#==========================================================================
def read_labeled_image_list(image_list_file, training_img_dir):
    """Reads a .txt file containing pathes and labeles
    Args:
       image_list_file: a .txt file with one /path/to/image per line
       label: optionally, if set label will be pasted after each line
    Returns:
       List with all filenames in file image_list_file
    """
    f = open(image_list_file, 'r')
    filenames = []
    labels = []

    for line in f:
        filename, label = line[:-1].split(' ')
        filename = training_img_dir+filename
        filenames.append(filename)
        labels.append(int(label))
        
    return filenames, labels
    
    
def read_images_from_disk(input_queue, size1=256):
    """Consumes a single filename and label as a ' '-delimited string.
    Args:
      filename_and_label_tensor: A scalar string tensor.
    Returns:
      Two tensors: the decoded image, and the string label.
    """
    label = input_queue[1]
    fn=input_queue[0]
    file_contents = tf.read_file(input_queue[0])
    example = tf.image.decode_jpeg(file_contents, channels=3)
    
    #example = tf.image.decode_png(file_contents, channels=3, name="dataset_image") # png fo rlfw
    example=tf.image.resize_images(example, [size1,size1])
    return example, label, fn

def setup_inputs(sess, filenames, training_img_dir, image_size=256, crop_size=224, isTest=False, batch_size=64):
    
    # Read each image file
    image_list, label_list = read_labeled_image_list(filenames, training_img_dir)

    images = tf.cast(image_list, tf.string)
    labels = tf.cast(label_list, tf.int64)
     # Makes an input queue
    if isTest is False:
        isShuffle = True
        numThr = 4
    else:
        isShuffle = False
        numThr = 1
        
    input_queue = tf.train.slice_input_producer([images, labels], shuffle=isShuffle)
    image, y,fn = read_images_from_disk(input_queue)

    channels = 3
    image.set_shape([None, None, channels])
        
    # Crop and other random augmentations
    if isTest is False:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        image = tf.random_crop(image, [crop_size, crop_size, 3])
    else:
        image = tf.image.central_crop(image,(224/256))    
        
    
    image = tf.cast(image, tf.float32)/255.0
    
    image, y,fn = tf.train.batch([image, y, fn], batch_size=batch_size, capacity=batch_size*3, num_threads=numThr, name='labels_and_images')

    tf.train.start_queue_runners(sess=sess)

    return image, y, fn, len(label_list)


In [ ]:
def activation(x,name="activation"):
    return tf.nn.relu(x, name=name)
    
def conv2d(name, l_input, w, b, s, p):
    l_input = tf.nn.conv2d(l_input, w, strides=[1,s,s,1], padding=p, name=name)
    l_input = l_input+b

    return l_input

def batchnorm(conv, isTraining, name='bn'):
    return tf.layers.batch_normalization(conv, training=isTraining, name="bn"+name)

def initializer(in_filters, out_filters, name, ks=3):
    w1 = tf.get_variable(name+"W", [ks, ks, in_filters, out_filters], initializer=tf.truncated_normal_initializer())
    b1 = tf.get_variable(name+"B", [out_filters], initializer=tf.truncated_normal_initializer())
    return w1, b1

def target_num_channels(in_x,growth_rate,isTraining,name):
    target_num_channels = int(in_x.get_shape()[-1]) + 4 * growth_rate
    w1,b1 = initializer(int(in_x.get_shape()[-1]),target_num_channels,name+"target_num_channels",ks = 1)
    x = batchnorm(in_x, isTraining, name=name+'target_num_channels')
    x = activation(x)
    x = conv2d(name+'r1', x, w1, b1, 1, "SAME")
    return x
    

def block_unit(in_x, growth_rate,isTraining, name):
    """========================bottleneck========================"""
    x = target_num_channels(in_x , growth_rate,isTraining ,name)
    channels = int(x.get_shape()[-1])
    
    w1, b1 = initializer(channels, 4*growth_rate, name+"block_unit_unit1", ks=1)
    x = batchnorm(x, isTraining, name=name+'bottleneck_unit1')
    x = activation(x)
    x = conv2d(name+'r1', x, w1, b1, 1, "SAME")

    w2, b2 = initializer(int(x.get_shape()[-1]), growth_rate, name+"block_unit_unit2")
    x = batchnorm(x, isTraining, name=name+'bottleneck_unit2')
    x = activation(x)
    x = conv2d(name+'r2', x, w2, b2, 1, "SAME")
    """========================bottleneck========================"""
    return x
    
def transition_layer(in_x,isTraining, theta, name):
    channels = int(in_x.get_shape()[-1])
    x = batchnorm(in_x, isTraining, name=name+'tra')
    w1, b1 = initializer(channels, int(channels*theta), name+"transition_lay", ks=1)
    x = activation(x)
    x = conv2d(name+'r1', x, w1, b1, 1, "SAME")
    x = tf.nn.avg_pool(value=x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID', name=name+'avg_pool_2x2')
    return x

In [ ]:
def dense_block(in_x, num_layers ,growth_rate,isTraining, name):
    channels = int(in_x.get_shape()[-1])
    global layer_count
    if layer_count > 0 :
        print("======================transition=======================")
    with tf.name_scope("block1") as scope:
        x = block_unit(in_x, growth_rate, isTraining, name+"block1")
        x = tf.concat(values=[in_x, x], axis=3, name=name+'stack0')
        layer_count += 1
        print("Denseblock%d with channels %d"%(layer_count,x.get_shape().as_list()[-1]))
        
    for i in range(num_layers-1):
        name_scope = "block" + str(i+2)
        layer_count += 1
        with tf.name_scope(name_scope) as scope:
            out_x = block_unit(x, growth_rate, isTraining, name+ "block" +str(i+2))
            x = tf.concat(values=[x, out_x], axis=3, name= name +'stack' +str(i+1))
            print("Denseblock%d with channels %d"%(layer_count,x.get_shape().as_list()[-1]))
    return x

      
def DenseNet(_X, isTraining,growth_rate):
    global n_classes
    w1 = tf.get_variable("initW", [7, 7, 3, 96], initializer=tf.truncated_normal_initializer())
    b1 = tf.get_variable("initB", [96], initializer=tf.truncated_normal_initializer())
    x = conv2d('conv1', _X, w1, b1, 2, "VALID")
    x = tf.nn.max_pool(x , ksize = [1,3,3,1], strides=[1,2,2,1], padding = "VALID")
    
    num_blocks = 4
    num_layers = [4,4,4,4]
    for i in range(num_blocks):
        x =dense_block(x, num_layers[i],growth_rate, isTraining, "denb%d"%(i))
        x =transition_layer(x, isTraining, 0.5, "transb%d"%(i))
        

    channels = int(x.get_shape()[-1])
    x = batchnorm(x, isTraining, name='FinalBn')    
    x = activation(x)
    wo, bo=initializer(channels, n_classes, "FinalOutput")
    x = conv2d('final', x, wo, bo, 1, "SAME")
    
    x = tf.reduce_mean(x, [1,2])
    W = tf.get_variable("FinalW", [n_classes, n_classes], initializer=tf.truncated_normal_initializer())
    b = tf.get_variable("FinalB", [n_classes], initializer=tf.truncated_normal_initializer())
    
    out = tf.matmul(x, W) + b
                            

    return out

In [ ]:
batch_size = 120
display_step = 80
learning_rate = tf.placeholder(tf.float32)      # Learning rate to be fed
lr = 5e-3                          # Learning rate start

In [ ]:
# Setup the tensorflow...
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

print("Preparing the training & validation data...")
train_data, train_labels, filelist1, glen1 = setup_inputs(sess, "data/train.txt", "data/", batch_size=batch_size)
val_data, val_labels, filelist2, tlen1 = setup_inputs(sess, "data/val.txt", "data/",isTest=True, batch_size=batch_size)

max_iter = glen1*100
print("Preparing the training model with learning rate = %.5f..." % (lr))


with tf.variable_scope("DenseNet") as scope:
    pred = DenseNet(train_data, True, 32)
    scope.reuse_variables()
    valpred = DenseNet(val_data, False, 32)

with tf.name_scope('Loss_and_Accuracy'):
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        cost = tf.losses.sparse_softmax_cross_entropy(labels=train_labels, logits=pred)
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
    correct_prediction = tf.equal(tf.argmax(pred, 1), train_labels)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    correct_prediction2 = tf.equal(tf.argmax(valpred,1), val_labels)
    accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2, tf.float32))

In [ ]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()
train_accuracy = []
val_accuracy = []
sess.run(init)
step = 0

while (step * batch_size) < max_iter:
    epoch1=np.floor((step*batch_size)/glen1)
    if (((step*batch_size)%glen1 < batch_size) & (lr==5e-3) & (epoch1 >10)):
        lr /= 10
    if (((step*batch_size)%glen1 < batch_size) & (lr==5e-4) & (epoch1 >20)):
        lr /= 5
    if (((step*batch_size)%glen1 < batch_size) & (lr==1e-4) & (epoch1 >30)):
        lr /= 2
    if (((step*batch_size)%glen1 < batch_size) & (lr==5e-5) & (epoch1 >40)):
        lr /= 5
    if (((step*batch_size)%glen1 < batch_size) & (lr==1e-5) & (epoch1 >60)):
        lr /= 10
    if (((step*batch_size)%glen1 < batch_size) & (lr==1e-6) & (epoch1 >80)):
        lr /= 10         

    sess.run(optimizer,  feed_dict={learning_rate: lr})

    if step>1 and step % display_step == 1:
        # calculate the loss
        loss, acc = sess.run([cost, accuracy])
        print("Iter=%d/epoch=%d, Loss=%.6f, Training Accuracy=%.6f, lr=%f" % (step*batch_size, epoch1 ,loss, acc,  lr))
        train_accuracy.append(acc)        
    if step>1 and (step % (display_step*10) == 0):
        rounds = tlen1 // batch_size
        
        valacc=[]
        for k in range(rounds):
            acc= sess.run(accuracy2)
            valacc.append(acc)
        print("\nIter=%d/epoch=%d, Validation Accuracy=%.6f" % (step*batch_size, epoch1 , np.mean(valacc)))
        val_accuracy.append(np.mean(valacc))
        
    step += 1
print("Optimization Finished!")
save_path = saver.save(sess, "./tfdensenet_advance_model.ckpt")
print("Model saved in file: %s" % save_path)
sess.close()